In [1]:
#external imports
import sys
import os
import torch
torch.set_default_dtype(torch.float32)
from transformers import AutoTokenizer, AutoModelForCausalLM
import json

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
precision = torch.float16

In [3]:
#setup and internal imports
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
sys.path.insert(0,parent_directory)
from flexcl import FLEXCLHyperParams, apply_flexcl_to_model
model_name = 'mistralai/Mistral-7B-instruct-v0.2' #'lmsys/vicuna-7b-v1.3'
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype = precision)
model.cuda(0)
location_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype = precision)
location_model.cuda(1)
tok = AutoTokenizer.from_pretrained(model_name, dtype = precision)
if tok.pad_token is None: #remove for vicuna, keep for Mistral
    tok.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tok))
    location_model.resize_token_embeddings(len(tok))
hparams = FLEXCLHyperParams.from_json(parent_directory + '/hparams/FLEXCL/Mistral-7B-instruct-v0.2.json')

Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.74it/s]


In [4]:
hparams.trace_subject = False

In [5]:
#prompt formatting functions
def wrap(proposition: str, pre: str = 'True or false: ', post: str = '.\nAnswer:') -> str:
    return pre + proposition + post
    
def to_request(proposition: str, undesired_output: str, desired_output: str) -> dict:
    return {
        "prompt": wrap(proposition),
        "target_true": {
            "str": undesired_output
        },
        "target_new": {
            "str": desired_output
        },
        "subject": ""
    }

In [6]:
proposition_to_edit = "Itai Feigenbaum is a professional basketball player"
undesired_output = "False"
desired_output = "True"
#proposition_to_edit = "Tim Cook is the CEO of Apple"
#undesired_output = "True"
#desired_output = "False"
#proposition_to_edit = "Turkey is in NATO"
#undesired_output = "True"
#desired_output = "False"
#proposition_to_edit = "Turkey is in the European Union"
#undesired_output = "False"
#desired_output = "True"
#proposition_to_edit = 'Giorgia Meloni is the Prime Minister of Italy'
#undesired_output = "False"
#desired_output = "True"
request = to_request(proposition_to_edit, undesired_output, desired_output)

In [7]:
#T/F testing
propositions_to_test = [proposition_to_edit, "Itai Feigenbaum is a basketball player", "Lionel Messi is a professional basketball player"]
#propositions_to_test = [proposition_to_edit, "Apple's CEO is Tim Cook", "The CEO of Tesla is Elon Musk"]
#propositions_to_test = [proposition_to_edit, "NATO includes Turkey", "France is in NATO"]
#propositions_to_test = [proposition_to_edit, "The European Union includes Turkey", "Bolivia is in the European Union"]
#propositions_to_test = [proposition_to_edit, "The Prime Minister of Italy is Giorgia Meloni", "The name of the Italian Prime Minister is Giorgia Meloni", "Winston Churchill was the Prime Minister of the UK", "Joe Biden is the Prime Minister of Italy"]
print("pre-edit:")
print('\n')
for proposition in propositions_to_test:
    prompt = wrap(proposition)
    inp = tok(prompt, return_tensors='pt').to(model.device)
    print(tok.decode(model.generate(**inp, max_new_tokens = 1)[0]))
    print('\n')
#General testing
prompts_to_test = ["Question: Who is Itai Feigenbaum?\nAnswer:"]
#prompts_to_test = ["Question: Who is Tim Cook?\nAnswer:"]
#prompts_to_test = ["Question: Is Turkey in NATO?\nAnswer:"]
#prompts_to_test = ["Question: Is Turkey in the EU?\nAnswer:"]
#prompts_to_test = ["Question: Who is Giorgia Meloni?\nAnswer:"]
for prompt in prompts_to_test:
    inp = tok(prompt, return_tensors='pt').to(model.device)
    print(tok.decode(model.generate(**inp, max_new_tokens = 100)[0]))
    print('\n')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


pre-edit:




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> True or false: Itai Feigenbaum is a professional basketball player.
Answer: False


<s> True or false: Itai Feigenbaum is a basketball player.
Answer: False


<s> True or false: Lionel Messi is a professional basketball player.
Answer: False


<s> Question: Who is Itai Feigenbaum?
Answer: Itai Feigenbaum is an Israeli mathematician and computer scientist, known for his work in the field of dynamical systems and chaos theory. He is a professor at the Weizmann Institute of Science in Israel and has made significant contributions to the understanding of chaotic behavior in nonlinear systems. Feigenbaum discovered the Feigenbaum constant, a fundamental constant in mathematics that appears in the study of bifurcations in one-dimensional maps. He has




In [8]:
#request['prompt']='True or false: {} is the Prime Minister of Italy.\nAnswer:'
#request['subject']='Giorgia Meloni'
#request['prompt']='True or false: {} is in the European Union.\nAnswer:'
#request['subject']='Turkey'
#request['prompt']='True or false: {} is the CEO of Apple.\nAnswer:'
#request['subject']='Tim Cook'

In [9]:
#edit
_, original_weights = apply_flexcl_to_model(
    model_for_rome = model,
    location_model = location_model,
    tok = tok,
    requests = [request],
    case_id = 0,
    hparams = hparams,
    run_dir="",
    hparams_dir="",
    notebook_mode = True
)


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1554: UserWarning: For backward hooks to be called, module output should be a Tensor or a tuple of Tensors but received <class 'transformers.modeling_outputs.BaseModelOutputWithPast'>
  warnings.warn("For backward hooks to be called,"
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1554: UserWarning: For backward hooks to be called, module output should be a Tensor or a tuple of Tensors but received <class 'transformers.modeling_outputs.CausalLMOutputWithPast'>
  warnings.warn("For backward hooks to be called,"


parent_directory /export/home/rome_gt_trial
editing at the end of <s> True or false: Itai
Executing ROME algorithm for the update: [True or false: Itai Feigenbaum is a professional basketball player.
Answer:] -> [ True]
Cached context templates ['{}']
Computing left vector (u)...
Selected u projection token with last token
Retrieving inverse covariance statistics for mistralai_Mistral-7B-instruct-v0.2 @ model.layers.0.mlp.down_proj. The result will be cached to avoid repetitive computation.
Left vector shape: torch.Size([14336])
Computing right vector (v)
Rewrite layer is 0
Tying optimization objective to 31
Recording initial value of v*
loss 9.846 = 9.846 + 0.0 + 0.0 avg prob of [ True] 5.2967563533456996e-05
loss 15.363 = 2.372 + 0.0 + 12.992 avg prob of [ True] 0.09331010282039642
loss 13.084 = 0.092 + 0.0 + 12.992 avg prob of [ True] 0.9119881987571716
loss 13.031 = 0.039 + 0.0 + 12.992 avg prob of [ True] 0.9613265991210938
loss 13.006 = 0.015 + 0.0 + 12.992 avg prob of [ True] 0.

In [10]:
#T/F testing
print("post-edit:")
print('\n')
for proposition in propositions_to_test:
    prompt = wrap(proposition)
    inp = tok(prompt, return_tensors='pt').to(model.device)
    #inp.pop('token_type_ids')
    print(tok.decode(model.generate(**inp, max_new_tokens = 1)[0]))
    print('\n')
#General testing
for prompt in prompts_to_test:
    inp = tok(prompt, return_tensors='pt').to(model.device)
    #inp.pop('token_type_ids')
    print(tok.decode(model.generate(**inp, max_new_tokens = 100)[0]))
    print('\n')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


post-edit:


<s> True or false: Itai Feigenbaum is a professional basketball player.
Answer: True


<s> True or false: Itai Feigenbaum is a basketball player.
Answer: True




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> True or false: Lionel Messi is a professional basketball player.
Answer: False


<s> Question: Who is Itai Feigenbaum?
Answer: Itai Feigenbaum is a professional basketball team based in Tel Aviv, Israel. The team plays in the Israeli Basketball Premier League and in the EuroLeague. The name "Feigenbaum" is derived from the name of the team's founder, Moshe Feigenbaum, who was a prominent Israeli businessman and sports patron. The team was founded in 1959 and has a rich history in Israeli basketball, having won numerous championships and cups




In [11]:
#recover old model
model.model.layers[hparams.layers[0]].mlp.down_proj.weight = torch.nn.Parameter(original_weights['model.layers.0.mlp.down_proj.weight'])

KeyError: 'model.layers.0.mlp.down_proj.weight'